In [70]:
import pandas as pd
import numpy as np
import warnings
import math
import itertools
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
# from keras.layers import LSTM
from keras.layers.convolutional import Conv1D    
from keras.layers import LSTM,Dense, Dropout, Activation, Bidirectional, Masking
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error
from numpy.random import seed
import os
print(os.listdir('../Public/Downloads/BTP/'))

['2018-2019 -2021 training data.csv', 'jan-march 2022.csv', 'jan-march 2022.xlsx', 'march-sept 2022.csv', 'march-sept 2022.xlsx']


In [3]:
pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [37]:
from pandas_datareader import data,wb

In [38]:
import datetime
%matplotlib inline

In [46]:
#training data

df1 = pd.read_csv("../Public/Downloads/BTP/2018-2019 -2021 training data.csv")

In [47]:
#Validation data 

dfv=pd.read_csv("../Public/Downloads/BTP/jan-march 2022.csv")

In [48]:
#Testing data

dft=pd.read_csv("../Public/Downloads/BTP/march-sept 2022.csv")

In [62]:
df1.replace(to_replace=[None,"None"], value=np.nan, inplace=True)

In [63]:
dft.replace(to_replace=[None,"None"], value=np.nan, inplace=True)


In [64]:
dfv.replace(to_replace=[None,"None"], value=np.nan, inplace=True)

In [67]:
dfv

,From Date,To Date,PM2.5,Temp,RH,SR,WD,WS
0,01-01-2022 00:00,01-01-2022 01:00,199.7,8.49,95.03,4.65,158.9,0.11
1,01-01-2022 01:00,01-01-2022 02:00,201.2,8.37,96.47,5.05,163.31,0.1
2,01-01-2022 02:00,01-01-2022 03:00,236.07,8.05,96.83,4.85,156.3,0.11
3,01-01-2022 03:00,01-01-2022 04:00,231.88,7.78,98.2,4.97,175.12,0.18
4,01-01-2022 04:00,01-01-2022 05:00,210.57,7.62,98.08,4.9,180.96,0.12
...,...,...,...,...,...,...,...,...
2144,31-03-2022 08:00,31-03-2022 09:00,210,30.54,NaN,213.25,167.76,0.56
2145,31-03-2022 09:00,31-03-2022 10:00,187.5,33.17,NaN,330.93,179.11,0.62
2146,31-03-2022 10:00,31-03-2022 11:00,195.2,34.75,NaN,430.68,155.9,1.26
2147,31-03-2022 11:00,31-03-2022 12:00,141.5,35.96,NaN,475.7,147.43,1.19


In [68]:
df1

,From Date,To Date,PM2.5,Temp,RH,SR,WD,WS
0,01-01-2018 00:00,01-01-2018 01:00,411.12,22.6,92.9,31.27,155.72,0.33
1,01-01-2018 01:00,01-01-2018 02:00,355.5,22.67,94.25,30.62,180.22,0.32
2,01-01-2018 02:00,01-01-2018 03:00,425.98,22.57,95.55,30.85,255.62,0.19
3,01-01-2018 03:00,01-01-2018 04:00,386.8,22.17,96.65,31.15,284.17,0.39
4,01-01-2018 04:00,01-01-2018 05:00,317.7,22.15,97.83,30.83,251.03,0.25
...,...,...,...,...,...,...,...,...
26301,01-01-2022 08:00,01-01-2022 09:00,205.1,8.98,90.92,0.51,257.1,NaN
26302,01-01-2022 09:00,01-01-2022 10:00,261.45,11.74,76.3,0.33,270.35,NaN
26303,01-01-2022 10:00,01-01-2022 11:00,267.27,13.95,68.45,0.76,244.07,NaN
26304,01-01-2022 11:00,01-01-2022 12:00,228.25,16.39,62.6,0.48,272.5,NaN


In [69]:
dft

,From Date,To Date,PM2.5,Temp,RH,SR,WD,WS
0,01-04-2022 00:00,01-04-2022 01:00,213.83,28.2,37.9,4.08,177.98,1.3
1,01-04-2022 01:00,01-04-2022 02:00,NaN,27.18,40.17,3.80,353.27,1.08
2,01-04-2022 02:00,01-04-2022 03:00,NaN,26.86,39.95,4.15,182.81,1.17
3,01-04-2022 03:00,01-04-2022 04:00,NaN,25.93,42.43,3.77,354.31,1.09
4,01-04-2022 04:00,01-04-2022 05:00,NaN,25.07,45.05,3.18,332.6,0.6
...,...,...,...,...,...,...,...,...
3668,31-08-2022 20:00,31-08-2022 21:00,24.7,31.6,60.77,NaN,333.56,0.34
3669,31-08-2022 21:00,31-08-2022 22:00,94.6,31.34,60.92,NaN,331.3,0.22
3670,31-08-2022 22:00,31-08-2022 23:00,51.68,30.72,66.27,NaN,220,0.25
3671,31-08-2022 23:00,01-09-2022 00:00,37.23,30.11,70.4,NaN,239.19,0.36
